In [ ]:
import MySQLdb
import pandas as pd
import numpy as np
from multiprocessing import Process
import time

In [ ]:
def crawl(portion_cnt, portion_step, total_end):
    db = MySQLdb.connect(host="192.168.120.127", user="reader", passwd="trreadonly", db="city_data", port=3306, charset="utf8")
    start = portion_cnt*portion_step
    end = min((portion_cnt+1)*portion_step, total_end)
    print("crawling {0} to {1}".format(start, end))
    cursor = db.cursor()
    cursor.execute("SELECT distinct(dev_id),dev_name FROM (SELECT * FROM city_data.city_data LIMIT {0} OFFSET {1}) as a;".format(portion_step, start))
    a = cursor.fetchall()
    pd.DataFrame(np.array(a), columns=["dev_id", "dev_name"]).to_csv("list_dev_id_part_{0}.csv".format(portion_cnt))
    db.close()

In [ ]:
def check_all_workers_working(list_cur_p):
    for i in range(len(list_cur_p)):
        if not list_cur_p[i].is_alive():
            return i

    return -1

In [ ]:
total_rows = 340421468
portion_step = 500000
total_rounds = total_rows//portion_step + 1
n_workers = 8

process_list = []

for i in range(total_rounds):
    p = Process(target=crawl, 
                args=(i, portion_step, total_rows))
    process_list.append(p)
    

i = 0
list_cur_p = []
for p in process_list:
    if len(list_cur_p) < n_workers:
        print(i)
        p.start()
        list_cur_p.append(p)
        i += 1
    if len(list_cur_p) < n_workers:
        continue

    idle = check_all_workers_working(list_cur_p)

    while idle == -1:
        time.sleep(1)
        idle = check_all_workers_working(
            list_cur_p)
    del list_cur_p[idle]

for p in list_cur_p:
    p.join()
    
print("finished")

In [ ]:
# import chardet
# print(dir(chardet))
# chardet.detect(a[0][1].encode())